In [59]:
import csv
import numpy as np
from sklearn import linear_model, preprocessing
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold

In [60]:
def E_abs(y_predict,y):
    E=0
    for i in range(len(y)):
        E+=abs(y_predict[i]-y[i])
    return E/len(y)

def E_01(y_predict,y):
    y_temp=[]
    for i in range(len(y)):
        if y_predict[i]>=0.5:
            y_temp.append(1)
        else:
            y_temp.append(0)
    return E_abs(y_temp,y)

In [61]:
#read training features
x=[]
temp=0
with open("../../data/ML_final_project/sample_train_x.csv") as f:
    for row in csv.reader(f):
        if temp==0:
            for i in range(len(row)): 
                x.append([])
            temp+=1
        else:
            for i,value in enumerate(row):
                x[i].append(float(value))
    f.close()
x=np.array(x)
x=np.transpose(x)

In [62]:
#read training truth
y=[]
with open('../../data/ML_final_project/truth_train.csv') as f:
    reader=csv.reader(f)
    for row in reader:
        for i, value in enumerate(row):
            if i!=0:
                y.append(float(value))
    f.close()
y=np.array(y)

In [63]:
#preprocessing the training data
# preprocessing.normalize(x, norm='l2')
# min_max_scaler = preprocessing.MinMaxScaler()
# x=min_max_scaler.fit_transform(x)

In [64]:
#training
n_folds=10
c_list=list(range(-4,5))
c_list[:]=[10 ** x for x in c_list]
n_components_list=range(10,18)
best_Eval_01=1
for c in c_list:
    for n_components in n_components_list:
        Ein_01=[]
        Ein_abs=[]
        Eval_01=[]
        Eval_abs=[]
        kf=KFold(len(x),n_folds=n_folds)
        for train_index, val_index in kf:
            x_train, x_val = x[train_index], x[val_index]
            y_train, y_val = y[train_index], y[val_index]
            pca = PCA(n_components=n_components)
            pca.fit(x_train)
            x_train_reduce = pca.transform(x_train)
            logistic = linear_model.LogisticRegression(C=c)
            logistic.fit(x_train_reduce,y_train)
            y_train_predict = logistic.predict(x_train_reduce)
            Ein_01.append(E_01(y_train_predict,y_train))
            Ein_abs.append(E_abs(y_train_predict,y_train))
            x_val_reduce = pca.transform(x_val)
            y_val_predict = logistic.predict(x_val_reduce)
            Eval_01.append(E_01(y_val_predict,y_val))
            Eval_abs.append(E_abs(y_val_predict,y_val))
        if sum(Eval_01)/n_folds<best_Eval_01:
            best_Eval_01=sum(Eval_01)/n_folds
            best_n_components=n_components
            best_c=c
print("best_Eval_01:",best_Eval_01,"best_n_components",best_n_components,"best_c:",best_c)
pca = PCA(n_components=best_n_components)
pca.fit(x)
x_reduce = pca.transform(x)
logistic = linear_model.LogisticRegression(C=best_c)
logistic.fit(x_reduce,y)

best_Eval_01: 0.142717462606 best_n_components 15 best_c: 0.1


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [66]:
#read testing data
x_test=[]
temp=0
with open('../../data/ML_final_project/sample_test_x.csv') as f:
    for row in csv.reader(f):
        if temp==0:
            for i in range(len(row)): 
                x_test.append([])
            temp+=1
        else:
            for i,value in enumerate(row):
                x_test[i].append(float(value))
    f.close()
x_test=np.array(x_test)
x_test=np.transpose(x_test)

In [67]:
#predict testing data
x_test_reduce=pca.transform(x_test)
y_test_predict = logistic.predict(x_test_reduce)

In [82]:
output_track1=np.vstack((x_test[:,0],y_test_predict))
output_track1=np.transpose(output_track1)
output_track2=np.copy(output_track1)
for i in range(len(output_track2)):
    if output_track2[i][1]>=0.5:
        output_track2[i][1]=1
    else:
        output_track2[i][1]=0

In [84]:
#write predict result to file
with open("../../result/PCA_logisticReg/test_track1.csv","w") as f:
    w=csv.writer(f)
    w.writerows(output_track1)
    f.close()
with open("../../result/PCA_logisticReg/test_track2.csv","w") as f:
    w=csv.writer(f)
    w.writerows(output_track2)
    f.close()